In [1]:
import pytse_client as tse
import numpy as np
import torch
import torch.nn as nn
# tickers = tse.download(symbols="all", write_to_csv=True)

Device = ("cuda"
          if torch.cuda.is_available()
          else "cpu"
)
print(f"Using {Device} device")


Using cuda device


In [2]:
sym="آپ"
ticker=tse.Ticker(sym)
length=ticker.history.shape[0]
train_length = int(np.floor(length*0.90))
test_length = length - train_length
days_history=50
days_predict=8

# Now we extract some portion of the first sequences of data for train
X_train=np.zeros([train_length - days_history - days_predict , days_history,5])
Y_train=np.zeros([train_length - days_history - days_predict , days_predict])
for i in range(train_length - days_history - days_predict):
    m = ticker.history.loc[i : i + days_history - 1].to_numpy()
    X_train[i,:,:] = np.array([(m[:,9]-m[:,8])/m[:,8] * 100 , m[:,7]/1e3 , m[:,6]/1e6 , m[:,2] , m[:,3]]).T
    m = ticker.history.loc[i + days_history : i + days_history - 1 + days_predict].to_numpy()
    Y_train[i,:] = (m[:,9]-m[:,8])/m[:,8] * 100

# Now we extract some portion of the last sequences of data for test
X_test=np.zeros([test_length - days_predict , days_history,5])
Y_test=np.zeros([test_length - days_predict , days_predict])
for i in range(test_length - days_predict):
    j = i + train_length - days_history
    m = ticker.history.loc[j : j + days_history - 1].to_numpy()
    X_test[i,:,:] = np.array([(m[:,9]-m[:,8])/m[:,8] * 100 , m[:,7]/1e3 , m[:,6]/1e6 , m[:,2] , m[:,3]]).T
    m = ticker.history.loc[j + days_history : j + days_history - 1 + days_predict].to_numpy()
    Y_test[i,:] = (m[:,9]-m[:,8])/m[:,8] * 100

train_arr = np.arange(train_length - days_history - days_predict - 1)
train_idx = np.random.permutation(train_arr)
X_train = X_train[train_idx]
Y_train = Y_train[train_idx]

test_arr = np.arange(test_length - days_predict - 1)
test_idx = np.random.permutation(test_arr)
X_test = X_test[test_idx]
Y_test = Y_test[test_idx]
X_train = torch.from_numpy(X_train).to(device=Device,dtype=torch.float32)
Y_train = torch.from_numpy(Y_train).to(device=Device,dtype=torch.float32)

X_test = torch.from_numpy(X_test).to(device=Device,dtype=torch.float32)
Y_test = torch.from_numpy(Y_test).to(device=Device,dtype=torch.float32)

In [3]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        n=64
        super().__init__()
        self.Lin1 = nn.Linear(8*input_size,2*n)
        self.Lin2 = nn.Linear(2*n,4*n)
        self.Lin3 = nn.Linear(4*n,2*n)
        self.Lin4 = nn.Linear(2*n,n)
        self.Lin5 = nn.Linear(n,output_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        #self.activation2 = nn.Tanh()
        self.drop = nn.Dropout(p=0.5)
        self.conv1 = nn.Conv1d(1,8,padding=4,kernel_size=9)
        self.conv2 = nn.Conv1d(8,16,padding=4,kernel_size=9)
        self.conv3 = nn.Conv1d(16,8,padding=4,kernel_size=9)
        self.flat = nn.Flatten(start_dim=1)

    def forward(self, x):
        x = self.conv1(x)
        for i in range(1):
            x_ = x
            x = self.conv2(x)
            x = self.activation1(x)
            x = self.drop(x) ###
            x = self.conv3(x)
            x = self.activation1(x)
            x = self.drop(x) ###
            x = x + x_
        
        x = self.flat(x)
        x = self.Lin1(x)
        x = self.activation1(x)
        x = self.Lin2(x)
        x = self.activation1(x)
        x = self.Lin3(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        x = self.Lin4(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        x = self.Lin5(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        return x

model=Model(days_history*5,days_predict).to(device=Device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
LossFn = nn.MSELoss()


In [4]:
batch_size = 40
train_Losses=[]
test_Losses=[]
N_train = int(np.floor(X_train.size(0)/batch_size))
N_test = int(np.floor(X_test.size(0)/batch_size))

for epoch in range(300):
    model.train()
    train_loss=torch.tensor(0,dtype=torch.float32)
    for i in range(N_train):
        x = torch.flatten(X_train[i*batch_size:(i+1)*batch_size,:,:],start_dim=1).unsqueeze(dim=1)
        y = Y_train[i*batch_size:(i+1)*batch_size,:]
        optimizer.zero_grad()
        out = model(x)
        loss_train = LossFn(y,out)
        loss_train.backward()
        optimizer.step()
        train_loss += loss_train.detach().item()
    train_Losses.append( train_loss/N_train )
   
    model.eval()
    test_loss=torch.tensor(0,dtype=torch.float32)
    test_acc=torch.tensor(0,dtype=torch.float32)
    with torch.no_grad():
        for i in range(N_test):
            x = torch.flatten(X_test[i*batch_size:(i+1)*batch_size,:,:],start_dim=1).unsqueeze(dim=1)
            y = Y_test[i*batch_size:(i+1)*batch_size,:].detach()
            out = model(x)
            loss_test = LossFn(y,out)
            test_loss += loss_test.detach().item()
            temp=y-out
            test_acc += torch.sum(1 - torch.abs(y-out)/2).detach().item()
    test_Losses.append( test_loss/N_test )
    accuracy = test_acc / (N_test*batch_size*days_predict) *100
    print(f'{epoch}: train loss is {train_Losses[epoch]} & test loss is {test_Losses[epoch]} & accuracy is %{accuracy}')

c:\Users\IT  CITY\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


0: train loss is 7923.12646484375 & test loss is 10.700337409973145 & accuracy is %-31.782133102416992
1: train loss is 166.1512451171875 & test loss is 5.553189277648926 & accuracy is %10.318767547607422
2: train loss is 59.54113006591797 & test loss is 4.9153523445129395 & accuracy is %15.433857917785645
3: train loss is 42.123809814453125 & test loss is 4.851239204406738 & accuracy is %15.708611488342285
4: train loss is 37.238521575927734 & test loss is 4.8975958824157715 & accuracy is %15.503625869750977
5: train loss is 28.245040893554688 & test loss is 4.832118988037109 & accuracy is %15.792240142822266
6: train loss is 21.303571701049805 & test loss is 4.839232921600342 & accuracy is %15.43858528137207
7: train loss is 19.3394718170166 & test loss is 4.821220874786377 & accuracy is %15.816805839538574
8: train loss is 17.950437545776367 & test loss is 4.841944694519043 & accuracy is %15.307040214538574
9: train loss is 16.951169967651367 & test loss is 4.854093074798584 & accur

KeyboardInterrupt: 